In [3]:
!curl -L https://bit.ly/2Wi4OXo -o little_prince.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   150  100   150    0     0    637      0 --:--:-- --:--:-- --:--:--   641

100    67    0    67    0     0    125      0 --:--:-- --:--:-- --:--:--   125

100   297  100   297    0     0    311      0 --:--:-- --:--:-- --:--:--   311
100   297  100   297    0     0    310      0 --:--:-- --:--:-- --:--:--     0

100 76620  100 76620    0     0  52939      0  0:00:01  0:00:01 --:--:-- 52939


In [87]:
from keras.layers import SimpleRNN
from keras.layers import Dense, Activation
from keras.models import Sequential
import numpy as np

In [38]:
input_file = './little_prince.txt'

In [39]:
fin = open(input_file, 'rb')
lines = []
for line in fin:
    line = line.strip().lower()  # strip 양옆 공백 제거  lower 소문자 치환
    line = line.decode("euckr","ignore") # 문자열을 EUC-KR 인코딩으로 디코딩합니다. 만약 디코딩 중 에러가 발생하면 무시합니다.
    if len(line) ==0:
        continue
    lines.append(line)
fin.close()

In [40]:
lines[:3]

['여섯 살 적에 나는 "체험한 이야기"라는 제목의, 원시림에 관한 책에서 기막힌 그림 하나를 본 적이 있다. 맹수를 집어삼키고 있는 보아 구렁이 그림이었다. 위의 그림은 그것을 옮겨 그린 것이다.',
 '그 책에는 이렇게 씌어 있었다.',
 '"보아 구렁이는 먹이를 씹지도 않고 통째로 집어삼킨다.그리고는 꼼짝도 하지 못하고 여섯 달 동안 잠을 자면서 그것을 소화시킨다."']

In [49]:
text = ' '.join(lines)

### 977라인의 글을 .join함수를 이용해 1개 라인 문장으로 합침 -> text
### text에는 977개의 음절이 있음
### set함수를 이용해 문자단위로 중복을 음절일 제거하니 850개의 문자가 있음 -> nb_chars

In [59]:
text[:50]

'여섯 살 적에 나는 "체험한 이야기"라는 제목의, 원시림에 관한 책에서 기막힌 그림 하나를'

In [60]:
len(lines)

977

In [61]:
len(text)

44802

In [62]:
text[:20]

'여섯 살 적에 나는 "체험한 이야기"'

In [63]:
chars = set(c for c in text) #  set() 함수를 사용하여 집합으로 만

In [64]:
nb_chars = len(chars)
nb_chars

850

In [65]:
list(chars)[:2]

['러', '요']

### 전체 문자열수와 중복제거 문자수

In [66]:
len(text), nb_chars

(44802, 850)

### 음절별 단어와 인덱스 번호를 추출해 딕셔너리변수에 담음 -> char2index, index2char

In [67]:
char2index = dict((c,i) for i, c in enumerate(chars))
index2char = dict((i,c) for i, c in enumerate(chars))

In [68]:
index2char[3]

'가'

In [69]:
text[:40]

'여섯 살 적에 나는 "체험한 이야기"라는 제목의, 원시림에 관한 책에서 '

In [70]:
index2char[0], char2index['가']

('러', 3)

In [71]:
for i in range(10):
    print(index2char[i])

러
요
둔
가
않
금
쑤
섬
풀
싸


In [73]:
seqlen =10
step=1

### 입력데이터 생성 : 1~10자리의 문자 -> input_chars
### 라벨데이터 생성 : 11번째 자리 문자 -> label_chars

In [76]:
input_chars = []
label_chars=[]
for i in range(0, len(text) - seqlen, step):
    input_chars.append(text[i: i+seqlen])
    label_chars.append(text[i + seqlen])

In [81]:
input_chars[110],label_chars[110]

(' 책에는 이렇게 씌', '어')

In [82]:
input_chars[:15]

['여섯 살 적에 나는',
 '섯 살 적에 나는 ',
 ' 살 적에 나는 "',
 '살 적에 나는 "체',
 ' 적에 나는 "체험',
 '적에 나는 "체험한',
 '에 나는 "체험한 ',
 ' 나는 "체험한 이',
 '나는 "체험한 이야',
 '는 "체험한 이야기',
 ' "체험한 이야기"',
 '"체험한 이야기"라',
 '체험한 이야기"라는',
 '험한 이야기"라는 ',
 '한 이야기"라는 제']

In [83]:
label_chars[:15]

[' ', '"', '체', '험', '한', ' ', '이', '야', '기', '"', '라', '는', ' ', '제', '목']

In [84]:
len(input_chars), len(label_chars)

(44792, 44792)

### 글자를 one-hot-encoding을 이용해 벡터로 변환
### 글자가 850개이므로 850개의 독립변수항목을 생성
### char2index을 이용해 벡터로 변환
### x[데이터 전체 행수,10개 ,독립변수 문자수]
### y[데이터 전체 행수,10개 ,종속변수 문자수]

In [90]:
x = np.zeros((len(input_chars), seqlen, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)

C:\Users\lj040\AppData\Local\Temp\ipykernel_22748\1978939750.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(input_chars), seqlen, nb_chars), dtype=np.bool)
C:\Users\lj040\AppData\Local\Temp\ipykernel_22748\1978939750.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)


In [91]:
x.shape, y.shape

((44792, 10, 850), (44792, 850))

In [95]:
for i, input_char in enumerate(input_chars):
    for j, char in enumerate(input_char):
        x[i, j, char2index[char]] = 1
        y[i, char2index[label_chars[i]]] = 1

In [96]:
x.shape, y.shape

((44792, 10, 850), (44792, 850))

In [102]:
len(x[0][0])

850

### RNN모델 설계

In [103]:
hidden_size = 128
batch_size = 128
num_iterations = 25
num_epochs_per_iterations =1
num_preds_per_epoch = 10

In [104]:
model = Sequential()
model.add(SimpleRNN(hidden_size,return_sequences=False, input_shape= (seqlen, nb_chars), unroll=True))
model.add(Dense(nb_chars))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               125312    
                                                                 
 dense (Dense)               (None, 850)               109650    
                                                                 
 activation (Activation)     (None, 850)               0         
                                                                 
Total params: 234,962
Trainable params: 234,962
Non-trainable params: 0
_________________________________________________________________


In [105]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

### 학습하기

In [114]:
for iteration in range(num_iterations):
    print( '=' * 50)
    print( 'Iteration #: %d' % (iteration))
    model.fit(x, y, batch_size=batch_size, epochs = num_epochs_per_iterations)
    
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print('입력 : %s' % (test_chars))
    print(test_chars, end='')
    
    for i in range(num_preds_per_epoch):
        xtest = np.zeros((1, seqlen, nb_chars))
    
        for i , ch in enumerate(test_chars):
            xtest[0, i, char2index[ch]] =1
        
            pred = model.predict(xtest, verbose=0)[0]
            ypred = index2char[np.argmax(pred)]  # 가장 확률이 큰 인덱스의 번호를
            print(ypred, end ='') # end 줄바꿈을 안함
    
            test_chars = test_chars[1:] + ypred
        print()

Iteration #: 0
350/350 [==============================] - 10s 27ms/step - loss: 1.8282
입력 :  싶습니다.....
 싶습니다.....        ..
      . ..
        ..
      . ..
        ..
      . ..
        ..
      . ..
        ..
      . ..
Iteration #: 1
350/350 [==============================] - 9s 24ms/step - loss: 1.7907
입력 : 때 아무것도 이해할
때 아무것도 이해할      어어  
    다    있
    니 ..지어
       .  
    다 ...그
    니 니.. 
       ...
    다 ....
    니 니...
       ...
Iteration #: 2
350/350 [==============================] - 8s 23ms/step - loss: 1.7553
입력 : 로등 켜는 사람이 
로등 켜는 사람이 " 이는이이이 이말
    다다다로다을
          
      ...있
      ....
      ....
      ....
      ....
      ....
      ....
Iteration #: 3
350/350 [==============================] - 9s 25ms/step - loss: 1.7216
입력 : 일을 해야 해. 아
일을 해야 해. 아라다 다다다다다다저
 고고고가고 해  
        을해
      .. .
       .가.
      .. .
       .가.
      .. .
       .가.
      .. .
Iteration #: 4
350/350 [==============================] - 9s 25ms/step - loss: 1.6861
입력 : 그가 물었다. "지
그가 물었다. "

입력 : 도만        
도만        

하도 다 다....
도도도     ..
하가도??..?..
도만도    " "
하가이이)..이이이
도만  가가가가만 
하가의도지하다는하하
도만리리시    고
하가시하 는에에에는
도만        
